In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [2]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [9]:
# Find unique values in a specific column
unique_values = data['Emotions'].unique()

In [10]:
unique_values

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [11]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [12]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [13]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [14]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [15]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [16]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/15
400/400 [==============================] - 36s 77ms/step - loss: 1.3783 - accuracy: 0.4643 - val_loss: 0.9521 - val_accuracy: 0.6881
Epoch 2/15
400/400 [==============================] - 31s 78ms/step - loss: 0.3843 - accuracy: 0.8806 - val_loss: 0.5532 - val_accuracy: 0.8172
Epoch 3/15
400/400 [==============================] - 31s 77ms/step - loss: 0.0632 - accuracy: 0.9827 - val_loss: 0.5438 - val_accuracy: 0.8303
Epoch 4/15
400/400 [==============================] - 32s 79ms/step - loss: 0.0216 - accuracy: 0.9956 - val_loss: 0.6331 - val_accuracy: 0.8209
Epoch 5/15
400/400 [==============================] - 33s 82ms/step - loss: 0.0151 - accuracy: 0.9974 - val_loss: 0.6686 - val_accuracy: 0.8209
Epoch 6/15
400/400 [==============================] - 30s 76ms/step - loss: 0.0143 - accuracy: 0.9973 - val_loss: 0.6710 - val_accuracy: 0.8206
Epoch 7/15
400/400 [==============================] - 32s 80ms/step - loss: 0.0105 - accuracy: 0.9976 - val_loss: 0.6995 - val_accuracy:

In [17]:
model_architecture = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_architecture)

model.save_weights("model_weights.h5")

In [18]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 232ms/step
['sadness']
